In [1]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot as plt
from pandas import Series
from datetime import datetime
from ipywidgets import IntProgress

/home/glissa/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


In [2]:
from math import sqrt
from warnings import catch_warnings
from warnings import filterwarnings
from fbprophet import Prophet
from tqdm import tqdm_notebook
import os

Importing plotly failed. Interactive plots will not work.


In [3]:
import warnings
warnings.filterwarnings('ignore')

### GLOBAL VARIABLES

In [4]:
INPUT_PATH = '../../../data/processed'
INPUT_FILE_NAME = 'dataproc_agg_v01'
OUTPUT_PATH = '../../../data/features'
OUTPUT_FILE_NAME = 'prophet_001'
DAYS_PRED = 28

### FUNCTIONS

In [5]:
# score a model, return None on failure
def score_model(train, cfg, period=DAYS_PRED):
    result = None
    

    m = Prophet(**cfg)


    # show all warnings and fail on exception if debugging
    with catch_warnings():
        filterwarnings("ignore")

        m.fit(train)
        future = m.make_future_dataframe(periods=period)

        forecast = m.predict(future)


    return forecast

### LOAD DATASET

In [6]:
data = pd.read_pickle(f'{INPUT_PATH}/{INPUT_FILE_NAME}.pkl')

In [7]:
data = data[data.part == 'train']

In [8]:
level = 'state_id'
y_true = 'demand_mean'
variables = [level,'date', y_true]
features = ['date', y_true]

In [9]:
data = data[variables]

### TRAIN MODEL

In [10]:
cfg_prophet = dict()

In [11]:
cfg_prophet['yearly_seasonality'] = True
cfg_prophet['daily_seasonality'] = False
cfg_prophet['seasonality_prior_scale'] = 0.1

In [12]:
preds = pd.DataFrame()

for idx in tqdm_notebook(data[level].unique()):
    train = data[data[level] == idx][features]
    train.columns = ['ds', 'y']
    
    pred = score_model(train, cfg_prophet)
    pred[level] = idx

    # append to the main preds dataset
    preds = preds.append(pred[-DAYS_PRED:])

    

In [13]:
features_prophet = [level, 'ds', 'trend', 'yhat_lower', 'yhat_upper', 'weekly',  'yearly',  'yhat']

In [14]:
preds = preds[features_prophet]

### SAVE RESULTS

In [20]:
preds.to_pickle(f'{OUTPUT_PATH}/{OUTPUT_FILE_NAME}.pkl')